In [1]:
from src.classes.network import Network
from src.classes.node import Node
from multiprocessing import Pool
from src.viusalization import plot_network, animate_network, plot_cascade_dist, plot_cascade_dist_average
from collections import defaultdict
import os, sys


#### Setting seed for fixed order for sets, for reproducability purposes

In [2]:
# %env PYTHONHASHSEED=134

In [3]:
def create_data(iters, network):

    all_cascade_sizes = []
    all_polarizations = []
    average_cascade_per_round = []
    average_polarization_per_round = []
    number_of_samplers = 20

    for _ in range(iters): 
        cascades, cascade_dist, cascade_polarization = network.analyze_network()
        average_cascade_per_round.append(sum(cascade_dist)/number_of_samplers)
        average_polarization_per_round.append(sum(cascade_polarization))
        all_cascade_sizes += cascade_dist
        all_polarizations += cascade_polarization

        # plot_network(network, cascades)

    data = defaultdict(list)
    for i, (size, polarization) in enumerate(zip(all_cascade_sizes, all_polarizations), 1):
        data[size].append(polarization)
    for size in data:
        data[size].sort()

    average_data = defaultdict(list)
    for (size, polarization) in zip(average_cascade_per_round, average_polarization_per_round):
        average_data[size].append(polarization) 
    for size in average_data: 
        average_data[size].sort()
        
    return data, average_data
    

In [ ]:
# voorbeeld

num_nodes = 200
correlation = -1
update_fraction = 0.1
starting_distribution = 0.5     # L / R ratio (niet per se nodig maar kan misschien leuk zijn om te varieern)
p = 0.01
k = 2
seedje = 32

network = Network(num_nodes, correlation, update_fraction, starting_distribution, seed=seedje, p=p)

number_of_iters=1000
data_before, average_data_before = create_data(number_of_iters, network)
largest_size = max(data_before.keys())
plot_cascade_dist_average(data_before, largest_size)


number_of_alterations = 0
for i in range(1000):
    network.update_round()
    # network.rng = np.random.default_rng(seedje)
    number_of_alterations += network.alterations
print(number_of_alterations)

data, average_data = create_data(number_of_iters, network)
plot_cascade_dist_average(data, largest_size)




In [ ]:
from src.classes.network import Network
from src.viusalization import animate_network

def create_network():
    num_nodes = 200
    correlation = -1
    update_fraction = 0.3
    starting_distribution = 0.5     # L / R ratio (niet per se nodig maar kan misschien leuk zijn om te varieern)
    p = 0.01
    k = 2


    netwerkje = Network(num_nodes, correlation, update_fraction, starting_distribution, seed=32, p=p)
    animation = animate_network(netwerkje)
create_network()


In [ ]:
from src.experimentation import experiment_a

pars = []
seed = 33
for i in range(10):
    seed += i*10
    netwerkje = Network(100, -1, p=0.02, seed=seed)
    pars.append(netwerkje)
    
PROCESSES = 10
assert PROCESSES < os.cpu_count(), "Lower the number of processes (PROCESSES)"
with Pool(PROCESSES) as pool:
        assert PROCESSES < os.cpu_count(), "Lower the number of processes (PROCESSES)"
        print(f"Starting parallel execution for {experiment_a} schedule")
        results = pool.map(experiment_a, pars)